In [ ]:
!pip install xlsxwriter

In [ ]:
from google.colab import drive
import os

# ==========================================
# 1. Force Mount Google Drive (Ensure reading the latest files)
# ==========================================
# force_remount=True is key! It forces a cache refresh,
# ensuring the IMS.xlsx you just saved on your computer is read immediately by Colab.
try:
    drive.unmount('/content/drive')
    print("✅ Google Drive unmount attempted, preparing to remount...")
except Exception as e:
    print(f"⚠️ Google Drive unmount failed or not needed: {e}")
drive.mount('/content/drive', force_remount=True)

# ==========================================
# 2. Lock Working Directory (Prevent file loss)
# ==========================================
target_folder_name = 'err'  # Your target folder
work_dir = f'/content/drive/MyDrive/{target_folder_name}'

# Check and create folder
if not os.path.exists(work_dir):
    print(f"⚠️ Folder '{target_folder_name}' does not exist, creating...")
    os.makedirs(work_dir)
else:
    print(f"📂 Folder '{target_folder_name}' already exists.")

# Change directory
try:
    os.chdir(work_dir)
    print(f"✅ Success! Directory changed to: {os.getcwd()}")
    print("   (Files forcibly refreshed, and new files will be saved here)")
except Exception as e:
    print(f"❌ Failed to change directory: {e}")

⚠️ Google Drive unmount failed or not needed: module 'google.colab.drive' has no attribute 'unmount'
Mounted at /content/drive
📂 Folder 'err' already exists.
✅ Success! Directory changed to: /content/drive/MyDrive/err
   (Files forcibly refreshed, and new files will be saved here)


In [ ]:
# ==========================================
# 🛠️ File Sync Checker (Sync Checker)
# ==========================================
import os
import datetime
import time

# 1. Re-confirm directory (Prevent failure to switch)
target_folder_name = 'err'
work_dir = f'/content/drive/MyDrive/{target_folder_name}'
if os.getcwd() != work_dir:
    try:
        os.chdir(work_dir)
        print(f"📂 Directory corrected to: {os.getcwd()}")
    except:
        print("⚠️ Unable to switch directory, please check path.")

# 2. Find the latest IMS file
files = [f for f in os.listdir('.') if f.endswith('.xlsx') and "IMS" in f and not f.startswith('~$')]
files.sort(key=lambda x: os.path.getmtime(x), reverse=True)

if files:
    target_file = files[0]
    # Get file timestamp
    timestamp = os.path.getmtime(target_file)
    file_time = datetime.datetime.fromtimestamp(timestamp)
    current_time = datetime.datetime.now()

    print(f"📄 Latest file seen by Colab: {target_file}")
    print(f"🕒 File modification time: {file_time.strftime('%Y-%m-%d %H:%M:%S')} (Colab time)")

    # Simple check: If file time is more than 10 minutes ago, it might be old
    time_diff = (current_time - file_time).total_seconds() / 60
    print(f"⏱️ Time since: {int(time_diff)} minutes ago")

    if time_diff > 10:
        print("\n⚠️ Warning: This file looks a bit old!")
        print("   👉 Please confirm your Google Drive desktop app shows 'Sync Complete' (Green Check).")
        print("   👉 Suggest waiting 10-30 seconds and clicking Run All again.")
else:
    print("❌ Error: No IMS Excel file found in current directory!")

📄 Latest file seen by Colab: IMS.xlsx
🕒 File modification time: 2026-01-06 21:54:14 (Colab time)
⏱️ Time since: 1327 minutes ago

⚠️ Warning: This file looks a bit old!
   👉 Please confirm your Google Drive desktop app shows 'Sync Complete' (Green Check).
   👉 Suggest waiting 10-30 seconds and clicking Run All again.


In [ ]:
%%writefile config.py
# ==========================================
# Configuration Center (Configuration v29.1)
# ==========================================
VERSION = "v29.1 (Feat: Overdue Auto-Correction)"

# Basic Parameters
DEFAULT_INTERVAL = 7       # Default replenishment cycle (days)
BLIND_SPOT_PENALTY = 1.5   # Penalty for stockouts
Z_SCORE = 1.28             # Safety stock coefficient (Service Level)
MIN_BUFFER_QTY = 0

# Threshold Parameters
HARD_CAP = 8               # Physical Hard Cap
LEGACY_CUTOFF_DATE = "2025-12-01" # [v28.4] Cutoff date for legacy strategy

# [v27.4] Smart Frequency Reduction Parameters
HOT_SES_THRESHOLD = 2.0    # Hot item definition (SES >= 2.0)
SAFE_COVERAGE_PCT = 0.80   # Safety coverage requirement (80%)
MAX_HOT_RISK_ALLOWED = 1   # Max tolerated hot item stockouts

# Rookie Phase Strategy
ROOKIE_THRESHOLD_TOTAL = 5 # Total rookie phase duration (visits)
PHASE_1_LIMIT = 2          # Phase 1 (Hard rule limit)

# Forecasting Parameters
SES_SPAN = 6               # SES Smoothing Span

# File Paths
SHADOW_LEDGER_FILE = "shadow_ledger.json"
SKU_HISTORY_FILE = "sku_history_tracker.json"
GRAVEYARD_FILE = "sku_graveyard.json"
VISIT_HISTORY_FILE = "visit_history.json"

Overwriting config.py


In [ ]:
%%writefile data_utils.py
import pandas as pd
import datetime
import re
import json
import os

# ==========================================
# Utilities
# ==========================================

def parse_strict_date(val):
    if isinstance(val, (datetime.datetime, pd.Timestamp)):
        return val.to_pydatetime() if isinstance(val, pd.Timestamp) else val
    val_str = str(val).strip().replace('.0', '')
    match = re.search(r'(\d+)', val_str)
    if not match: return None
    clean_str = match.group(1)
    if len(clean_str) == 7: clean_str = '0' + clean_str
    if len(clean_str) != 8: return None
    try: return datetime.datetime.strptime(clean_str, "%d%m%Y")
    except: return None

def load_json(filepath):
    if os.path.exists(filepath):
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                return json.load(f)
        except:
            return {}
    return {}

def save_json(filepath, data):
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
    except:
        pass

def load_sku_map(file_path):
    """
    [v27.2] Read SKU real categories from Excel
    """
    sku_map = {}
    try:
        xls = pd.ExcelFile(file_path)
        # Find sheet containing SKU
        sku_sheet = next((s for s in xls.sheet_names if 'sku' in s.lower()), None)

        if sku_sheet:
            # Read data (assuming no header or fixed structure)
            df = pd.read_excel(xls, sheet_name=sku_sheet, header=None)

            # Iterate from row 2 (skip header)
            # Col 0: Category (Snack, Chocolate, Bakery...)
            # Col 5: Name (Lay'S_Classic...)
            for idx, row in df.iloc[2:].iterrows():
                if pd.notna(row[5]) and pd.notna(row[0]):
                    name = str(row[5]).strip().lower()
                    raw_cat = str(row[0]).strip().lower()

                    # Mapping Logic
                    if 'bakery' in raw_cat:
                        final_cat = 'Bakery'
                    elif 'chocolate' in raw_cat or 'gummy' in raw_cat:
                        final_cat = 'Chocolate'
                    elif 'snack' in raw_cat:
                        final_cat = 'Chips'
                    else:
                        final_cat = 'Chips' # Default fallback

                    sku_map[name] = final_cat
            print(f"✅ SKU Map loaded successfully: Contains {len(sku_map)} products")
            return sku_map
    except Exception as e:
        print(f"⚠️ SKU Map load failed: {e}")

    return {}

def get_category_type(product_name, sku_map=None):
    """
    [v27.2] Prefer table lookup, fallback to guessing
    """
    name_lower = str(product_name).lower()

    # 1. Lookup (Precise)
    if sku_map and name_lower in sku_map:
        return sku_map[name_lower]

    # 2. Keyword Guessing (Fallback)
    if 'bakery' in name_lower or 'cookie' in name_lower: return 'Bakery'
    if 'chocolate' in name_lower or 'kit kat' in name_lower or 'mars' in name_lower: return 'Chocolate'
    return 'Chips'

Overwriting data_utils.py


In [ ]:
%%writefile storage_manager.py
import datetime
from data_utils import load_json, save_json
import config

# ==========================================
# Archivist (File Management)
# ==========================================

def update_shadow_ledger_entry(shadow_data, loc, sku, date_str, qty, cap):
    """Update Shadow Ledger: Record virtual stock and Cap settings"""
    if loc not in shadow_data: shadow_data[loc] = {}
    if sku not in shadow_data[loc]: shadow_data[loc][sku] = []

    found = False
    for entry in shadow_data[loc][sku]:
        if entry['date'] == date_str:
            entry['qty'] = qty
            entry['cap'] = cap
            found = True
            break

    if not found:
        shadow_data[loc][sku].append({'date': date_str, 'qty': qty, 'cap': cap})

def update_archives_and_save(shadow_ledger, advisor_data):
    """
    Update all backend archives:
    1. Shadow Ledger
    2. SKU History
    3. Graveyard
    4. Visit History
    """
    # 1. Save Shadow Ledger
    save_json(config.SHADOW_LEDGER_FILE, shadow_ledger)

    # Load other archives
    sku_history = load_json(config.SKU_HISTORY_FILE)
    graveyard = load_json(config.GRAVEYARD_FILE)
    visit_history = load_json(config.VISIT_HISTORY_FILE)

    today_str = datetime.datetime.now().strftime("%Y-%m-%d")

    for loc_name, loc_data in advisor_data.items():
        current_stats = loc_data['sku_stats']
        current_drop_qty = loc_data['total_drop_qty']

        # 2. Update Visit History (For frequency reduction logic next time)
        visit_history[loc_name] = {'last_drop_qty': current_drop_qty, 'date': today_str}

        # 3. Zombie Check (Graveyard Check)
        # If in old list but not in new list, it was removed
        if loc_name in sku_history:
            for old_sku, old_ses in sku_history[loc_name].items():
                if old_sku not in current_stats:
                    if loc_name not in graveyard: graveyard[loc_name] = []
                    # Avoid duplicate records for the same day
                    if not any(g['name'] == old_sku and g['date'] == today_str for g in graveyard[loc_name]):
                        graveyard[loc_name].append({
                            'name': old_sku,
                            'lost_ses': old_ses,
                            'date': today_str
                        })
                        print(f"⚰️  [Archive] {loc_name}: Detected removal of {old_sku} (Hist SES {old_ses:.2f})")

        # 4. Update SKU History
        # Overwrite old data with new SES
        sku_history[loc_name] = {k: v['ses'] for k, v in current_stats.items()}

    # Save all changes
    save_json(config.SKU_HISTORY_FILE, sku_history)
    save_json(config.GRAVEYARD_FILE, graveyard)
    save_json(config.VISIT_HISTORY_FILE, visit_history)

    print("✅ All background archives safely updated (Shadow, History, Graveyard, Visit).")

Overwriting storage_manager.py


In [ ]:
%%writefile advisor.py
import config

# ==========================================
# Operations Advisor (Advisor v29.0)
# ==========================================
class OperationsAdvisor:
    def __init__(self):
        pass

    def analyze(self, sku_stats, interval_days, is_rookie=False):
        """
        [v29.0] Only give advice for mature locations. Silence during rookie phase.
        """
        # 1. If rookie phase, return empty, keep quiet
        if is_rookie:
            return []

        advice_list = []
        total_skus = len(sku_stats)
        if total_skus == 0: return []

        safe_count = 0
        hot_risk_list = []

        for name, info in sku_stats.items():
            ses = info['ses']
            final_stock = info.get('final_stock', 0)

            daily_sales = ses / 7.0
            demand = daily_sales * interval_days

            is_safe = (final_stock >= demand) or (final_stock >= config.HARD_CAP)
            if is_safe: safe_count += 1

            if ses >= config.HOT_SES_THRESHOLD and not is_safe:
                hot_risk_list.append(f"{name}({ses:.1f})")

        coverage_pct = safe_count / total_skus
        hot_risk_count = len(hot_risk_list)

        # Only keep critical health info, minimize chatter
        risk_msg = "None" if hot_risk_count == 0 else f"{hot_risk_count} SKUs ({', '.join(hot_risk_list[:2])}...)"
        advice_list.append(f"📊 [Health] Coverage: {coverage_pct:.0%} | Risk: {risk_msg}")

        hot_candidates = {}
        cold_candidates = {}

        for name, info in sku_stats.items():
            cat = info['category']
            streak = info['stockout_streak']
            cap = info.get('current_cap', 4)
            ses = info['ses']

            # Report severe hot items
            if streak >= 2 and cap >= 8:
                if cat not in hot_candidates: hot_candidates[cat] = []
                hot_candidates[cat].append(name)
                advice_list.append(f"🔥 [Hot] {name}: Stockout {streak}x -> Suggest Adding Facings")

            # Report severe cold items (non-rookie)
            elif 0 < ses < 0.3 and streak == 0:
                advice_list.append(f"❄️ [Cold] {name}: Daily Sales only {ses:.2f} -> Suggest Delisting")

        return advice_list

Overwriting advisor.py


In [ ]:
%%writefile efficiency_checker.py
import config

# ==========================================
# Backtest Referee (Efficiency Checker v29.0)
# Responsibility: Review accuracy of the last replenishment
# ==========================================

class EfficiencyChecker:
    def __init__(self):
        self.total_sales = 0.0
        self.total_waste = 0.0  # Over-Carry
        self.total_missed = 0.0 # Under-Carry
        self.sku_count = 0

    def analyze_sku(self, last_real_record, ses_weekly, interval_days):
        """
        Analyze single SKU performance from last time
        last_real_record: contains {'stock'(Carry), 'rest', 'cons'}
        ses_weekly: Calculated smooth weekly sales (reference for potential)
        """
        if not last_real_record: return

        carried = last_real_record['stock'] # Amount carried last time (Cap)
        rest = last_real_record['rest']     # Amount remaining
        sales = last_real_record['cons']    # Actual sales

        # 1. Calculate Waste (Over-Carry)
        # Anything remaining is waste
        waste = rest

        # 2. Calculate Missed (Under-Carry)
        # Only calculate missed if stockout occurred (Rest=0)
        missed = 0.0
        if rest == 0:
            # Estimate real demand
            daily_demand = ses_weekly / 7.0
            total_demand = daily_demand * interval_days

            # Missed = Theoretical Demand - Actual Carried
            # If Theoretical < Carried (Random bulk buy), missed is 0
            missed = max(0, total_demand - carried)

        self.total_sales += sales
        self.total_waste += waste
        self.total_missed += missed
        self.sku_count += 1

    def get_score(self):
        """
        Output final score and stats
        Formula: Efficiency = Sales / (Sales + Waste + Missed)
        """
        if self.sku_count == 0:
            return None

        denominator = self.total_sales + self.total_waste + self.total_missed

        if denominator == 0:
            return {
                "score_pct": 0.0,
                "waste": self.total_waste,
                "missed": self.total_missed,
                "rating": "N/A" # All zeros
            }

        score = self.total_sales / denominator

        # Simple Rating
        if score >= 0.85: rating = "⭐ Excellent"
        elif score >= 0.70: rating = "✅ Good"
        elif score >= 0.50: rating = "⚠️ Fair"
        else: rating = "❌ Poor"

        return {
            "score_pct": score,
            "waste": self.total_waste,
            "missed": self.total_missed,
            "rating": rating
        }

Overwriting efficiency_checker.py


In [ ]:
%%writefile forecast_engine.py
import pandas as pd
import config

# ==========================================
# Forecast Engine
# Responsibility: Pure math, calculates SES and Std Dev
# ==========================================

def calculate_forecast(history, is_sparse):
    """
    Corresponds to original Layer 1 logic
    Calculates Exponential Weighted Moving Average (EWMA) and Standard Deviation
    """
    if not history: return 0.0, 0.0

    series = pd.Series(history)
    ewm_mean = series.ewm(span=config.SES_SPAN, adjust=False).mean().iloc[-1]
    std = series.std() if len(series) > 1 else 0.0

    # Sparse data / Stockout blind spot correction
    if is_sparse:
        max_reasonable_ses = (config.HARD_CAP * 1.5) / 7.0
        if ewm_mean < max_reasonable_ses:
            std *= config.BLIND_SPOT_PENALTY
            ewm_mean = min(ewm_mean * 1.2, max_reasonable_ses)

    return max(0, ewm_mean), std

Overwriting forecast_engine.py


In [ ]:
%%writefile policy_engine.py
import numpy as np
import config
from data_utils import get_category_type

# ==========================================
# Policy Engine (Policy Engine v28.7)
# Responsibility: Business rules, Rookie phase, Cap setting, Qty decision
# ==========================================

def apply_policy(p_name, mean, std, prev_cap, count, last_rest, sku_map, chip_idx_in_loc=0, is_legacy_mode=False):
    """
    [v28.7] Rookie Phase Logic Refactor: Simple promotion/relegation based on combat performance
    """
    cat = get_category_type(p_name, sku_map)
    policy = {}

    # ==========================================
    # 1. Highest Priority: Legacy Lock
    # ==========================================
    if is_legacy_mode and cat == 'Chips':
        policy = {
            "current_cap": prev_cap,
            "trigger": 999,
            "target": prev_cap,
            "is_rookie": False,
            "previous_cap": prev_cap,
            "note": "Legacy Lock"
        }
        return _apply_safety_net(policy, cat)

    # ==========================================
    # 2. Rookie Phase Strategy
    # ==========================================
    if count < config.ROOKIE_THRESHOLD_TOTAL:

        # --- Phase 1: Initial Layout (Visits 1-2) ---
        # Logic: Fixed by position (First 7 Chips get 8, rest 4)
        if count < config.PHASE_1_LIMIT:
            target = 4; current_cap = 4
            if cat == 'Chocolate': target = 8; current_cap = 8
            elif cat == 'Bakery': target = 4; current_cap = 4
            elif cat == 'Chips':
                if chip_idx_in_loc <= 6: target = 8; current_cap = 8
                else: target = 4; current_cap = 4
            policy = {"current_cap": current_cap, "trigger": 999, "target": target, "is_rookie": True, "previous_cap": prev_cap}

        # --- Phase 2: Combat Adjustment (Visits 3-5) [v28.7 New Logic] ---
        # Logic: Look at last performance, simple promotion/relegation
        else:
            # 1. Calculate last sales (approximate)
            # If last Cap was 8, rest 5, then sold 3
            last_sales = max(0, prev_cap - last_rest)

            # Default to inheriting last Cap
            new_cap = prev_cap

            # Only adjust Chips (Bakery/Choc locked by safety net later)
            if cat == 'Chips':
                # Rule A: Promotion (Cap 4 -> 8)
                # Condition: Last was 4 AND Stockout (Rest=0)
                if prev_cap == 4 and last_rest == 0:
                    new_cap = 8

                # Rule B: Relegation (Cap 8 -> 4)
                # Condition: Last was 8 AND Remaining(Rest) > Replenished/Sold(Sales)
                # Note: "Replenished" assumes "Sold" (assuming full refill last time)
                elif prev_cap == 8 and last_rest > last_sales:
                    new_cap = 4

                # Rule C: Maintain
                # (If conditions not met, new_cap remains prev_cap)

            # Set Target
            # Rookie phase fills to Cap
            target = new_cap
            current_cap = new_cap

            policy = {"current_cap": current_cap, "trigger": 999, "target": target, "is_rookie": True, "previous_cap": prev_cap}

    # ==========================================
    # 3. Mature Phase Strategy
    # ==========================================
    else:
        # Dynamic Cap Adjustment
        if cat == 'Chips':
             curr_cap = 8 if mean >= 2.0 else 4
        else:
             curr_cap = 4 # Default

        # Chips Relegation Protection (Retained in Mature Phase)
        if cat == 'Chips' and prev_cap > 0: pass
        else:
            if cat == 'Chips': prev_cap = 4

        buffer = (config.Z_SCORE * std) + config.MIN_BUFFER_QTY
        trigger = mean + buffer
        target = min(trigger, curr_cap)
        if target < 1: target = 1

        policy = {
            "current_cap": curr_cap,
            "trigger": trigger,
            "target": target,
            "is_rookie": False,
            "raw_forecast": mean,
            "previous_cap": prev_cap
        }

    # ==========================================
    # 4. Final Safety Net
    # ==========================================
    # This prevents Cookies from accidentally being set to 8, or Chocolate to 4
    return _apply_safety_net(policy, cat)

def _apply_safety_net(policy, cat):
    """
    Regardless of previous logic, apply final mandatory correction here.
    """
    # Bakery must be <= 4
    if cat == 'Bakery':
        policy['current_cap'] = 4
        if policy['target'] > 4: policy['target'] = 4

    # Chocolate must be <= 8
    elif cat == 'Chocolate':
        policy['current_cap'] = 8
        if policy['target'] > 8: policy['target'] = 8

    # Global Hard Cap
    if policy['target'] > config.HARD_CAP: policy['target'] = config.HARD_CAP

    return policy

def make_decision(start, is_fill, days, policy):
    # Rookie Phase or Legacy Lock -> Fill to Target directly
    if policy.get('is_rookie', False) or policy.get('note') == 'Legacy Lock':
        return int(np.ceil(policy['target'])), None, policy['current_cap']

    burn = policy['raw_forecast'] / 7.0
    est_stock = start - (burn * days)

    if est_stock > policy['trigger']:
        return 0, max(0, est_stock - policy['raw_forecast']), policy['previous_cap']

    return int(np.ceil(policy['target'])), None, policy['current_cap']

Overwriting policy_engine.py


In [ ]:
%%writefile core_logic.py
import pandas as pd
import datetime
from datetime import timedelta
import config
from data_utils import get_category_type, parse_strict_date
from storage_manager import update_shadow_ledger_entry
from advisor import OperationsAdvisor

# Import Engines
from forecast_engine import calculate_forecast
from policy_engine import apply_policy, make_decision
# [v29.0] Import Efficiency Checker
from efficiency_checker import EfficiencyChecker

# ==========================================
# Core Processor (Core Processor v29.1)
# ==========================================

def process_single_location_block(df_block, location_name, date_row, sku_map, loc_intervals, manual_plan, shadow_ledger, advisor_data_collector, current_interval=7, dry_run=False, target_date_override=None):
    # --- Date Parsing ---
    col_dates = {}
    for i in range(len(date_row)):
        raw_val = date_row.iloc[i]
        d = parse_strict_date(raw_val)
        if d:
            if i > 0: col_dates[i-1] = d
            col_dates[i] = d

    valid_dates = list(col_dates.values())
    valid_dates_set = set(d.strftime("%Y-%m-%d") for d in valid_dates)
    last_visit_date = max(valid_dates) if valid_dates else None
    last_visit_str = last_visit_date.strftime("%Y-%m-%d") if last_visit_date else "N/A"

    # [v29.1] Target Date Logic Update
    # Prefer override date from main.py (handling Overdue)
    # Fallback to default logic if not provided
    if target_date_override:
        target_date = target_date_override
    else:
        target_date = (last_visit_date + timedelta(days=7)) if last_visit_date else datetime.datetime.now()

    target_date_iso = target_date.strftime("%Y-%m-%d")

    # --- Legacy Strategy Detection ---
    legacy_map = {}
    col_indices = list(range(2, len(df_block.columns)-1, 2))
    valid_col_indices = [i for i in col_indices if col_dates.get(i)]

    try: legacy_cutoff = datetime.datetime.strptime(config.LEGACY_CUTOFF_DATE, "%Y-%m-%d")
    except: legacy_cutoff = datetime.datetime(2025, 12, 1)

    for i in valid_col_indices:
        curr_date = col_dates.get(i)
        is_legacy_day = True
        has_chips_data = False
        if curr_date and curr_date >= legacy_cutoff: is_legacy_day = False
        else:
            for idx in range(1, len(df_block)):
                row = df_block.iloc[idx]
                p_name = str(row.iloc[1])
                if pd.isna(p_name) or p_name == 'nan': continue
                cat = get_category_type(p_name, sku_map)
                c_val = row.iloc[i]
                if pd.notna(c_val):
                    try:
                        qty = float(c_val)
                        if cat == 'Chips':
                            has_chips_data = True
                            if qty != 2.0 and qty != 6.0:
                                is_legacy_day = False; break
                    except: continue
        if has_chips_data and is_legacy_day: legacy_map[i] = True
        else: legacy_map[i] = False

    # --- Main Loop Initialization ---
    results = []
    full_scan_list = []
    if location_name not in shadow_ledger: shadow_ledger[location_name] = {}
    current_loc_stats = {}
    total_drop_qty = 0
    period_stats = {}
    chip_counter = 0
    valid_dates_found = set()
    history_count = len(valid_col_indices)

    # [v29.0] Initialize Efficiency Checker
    efficiency_checker = EfficiencyChecker()
    is_rookie_loc = history_count < config.ROOKIE_THRESHOLD_TOTAL

    # --- Iterate Products ---
    for idx in range(1, len(df_block)):
        row = df_block.iloc[idx]
        p_name_raw = row.iloc[1]
        if pd.isna(p_name_raw): continue
        p_name = str(p_name_raw).strip()
        if p_name.lower() == 'nan' or p_name == "" or "Product" in p_name: continue

        timeline = []
        stockout_streak = 0
        cat_type = get_category_type(p_name, sku_map)
        if cat_type == 'Chips': chip_counter += 1

        sku_shadow_map = {}
        if p_name in shadow_ledger[location_name]:
            for entry in shadow_ledger[location_name][p_name]:
                sku_shadow_map[entry['date']] = entry

        sorted_indices = sorted(valid_col_indices, key=lambda x: col_dates[x])
        last_valid_col_idx = -1

        for k, i in enumerate(sorted_indices):
            c_val = row.iloc[i]; r_val = row.iloc[i+1]
            curr_date = col_dates.get(i)
            if pd.notna(c_val) and pd.notna(r_val):
                valid_dates_found.add(curr_date)
                last_valid_col_idx = i
                try:
                    c = float(c_val); r = float(r_val)
                    prev_date_iso = curr_date.strftime("%Y-%m-%d")

                    if legacy_map.get(i, False):
                        if cat_type == 'Chips': prev_cap = c
                        elif c == 4.0: prev_cap = 4.0
                        else: prev_cap = c
                    else:
                        if prev_date_iso in sku_shadow_map: prev_cap = sku_shadow_map[prev_date_iso].get('cap', c)
                        else: prev_cap = c

                    if r > prev_cap: cons = 0
                    else:
                        cons = max(0, prev_cap - r)
                        if r == 0: cons *= config.BLIND_SPOT_PENALTY

                    timeline.append({'date': curr_date, 'type': 'REAL', 'cons': cons, 'stock': prev_cap, 'rest': r})

                    if i not in period_stats:
                        days_span = 0
                        if k > 0:
                            prev_col_idx = sorted_indices[k-1]
                            prev_d = col_dates.get(prev_col_idx)
                            if prev_d: days_span = (curr_date - prev_d).days
                        period_stats[i] = {'sales': 0.0, 'days': days_span, 'date': curr_date}
                    period_stats[i]['sales'] += cons

                except ValueError: continue

        if p_name in shadow_ledger[location_name]:
            for entry in shadow_ledger[location_name][p_name]:
                try:
                    d = datetime.datetime.strptime(entry['date'], "%Y-%m-%d")
                    if entry['date'] in valid_dates_set: continue
                    if d < target_date: timeline.append({'date': d, 'type': 'VIRTUAL', 'cons': 0, 'stock': entry['qty'], 'rest': entry['qty'], 'cap': entry.get('cap', 4)})
                except: pass

        timeline.sort(key=lambda x: x['date'])
        for item in reversed(timeline):
            if item['type'] == 'REAL':
                if item['rest'] == 0: stockout_streak += 1
                else: break

        vel_history = []
        is_sparse = False
        if len(timeline) > 1:
            for i in range(1, len(timeline)):
                curr = timeline[i]; prev = timeline[i-1]
                days = (curr['date'] - prev['date']).days
                if days <= 0: continue
                daily_rate = (curr['cons']/days)
                if daily_rate > 3.0: daily_rate = 3.0
                vel_history.append(daily_rate * 7.0)
                if curr['type'] == 'VIRTUAL': is_sparse = True

        mean, std = calculate_forecast(vel_history, is_sparse)

        last = timeline[-1] if timeline else None
        prev_cap_for_policy = 4
        if last: prev_cap_for_policy = last.get('cap', last.get('stock', 4))
        start = last['stock'] if last else 0
        last_rest = last['rest'] if last else 0

        if last and last['type'] == 'REAL' and last['stock'] > 4:
            if not legacy_map.get(last_valid_col_idx, False): prev_cap_for_policy = 8

        is_legacy_mode = False
        if cat_type == 'Chips' and last_valid_col_idx != -1:
            if legacy_map.get(last_valid_col_idx, False): is_legacy_mode = True

        policy = apply_policy(p_name, mean, std, prev_cap_for_policy, len(vel_history), last_rest, sku_map, chip_counter, is_legacy_mode)

        # [v29.1] Critical Fix: 'days' passed here is for burn calculation
        # Use (target_date - last_visit_date) for real span
        # If overdue, target_date is today, days will be large, burn large, qty large
        days_diff = 7 # default
        if last and last['date']:
            days_diff = (target_date - last['date']).days
            if days_diff < 1: days_diff = 7

        qty, v_stock, res_cap = make_decision(start, True, days_diff, policy)

        if qty > config.HARD_CAP: qty = config.HARD_CAP

        final_stock_for_advisor = policy['target'] if qty > 0 else start
        current_loc_stats[str(p_name).strip()] = {
            'ses': mean,
            'category': cat_type,
            'stockout_streak': stockout_streak,
            'current_cap': policy['current_cap'],
            'final_stock': final_stock_for_advisor
        }

        full_scan_list.append([p_name, qty, policy['current_cap']])

        if not dry_run:
            update_shadow_ledger_entry(shadow_ledger, location_name, p_name, target_date_iso, v_stock if v_stock is not None else (start + qty), res_cap)
            if qty > 0:
                results.append([p_name, qty, policy['current_cap']])
                total_drop_qty += qty

        # [v29.0] Calculate backtest only after rookie phase
        if not is_rookie_loc and last and last['type'] == 'REAL':
             efficiency_checker.analyze_sku(last, mean, current_interval)

    # --- Loop End, Summary ---
    advisor_data_collector[location_name] = {'sku_stats': current_loc_stats, 'total_drop_qty': total_drop_qty}

    # [v29.0] Call Advisor, passing is_rookie_loc
    advisor = OperationsAdvisor()
    advice_messages = advisor.analyze(current_loc_stats, current_interval, is_rookie=is_rookie_loc)

    # [v29.0] Get Efficiency Score
    efficiency_score = efficiency_checker.get_score() if not is_rookie_loc else None

    sorted_periods_by_date = sorted(period_stats.values(), key=lambda x: x['date'], reverse=True)
    sales_history_list = []
    for p_data in sorted_periods_by_date:
        d_sales = 0.0
        if p_data['days'] > 0: d_sales = round(p_data['sales'] / p_data['days'], 2)
        sales_history_list.append(d_sales)
        if len(sales_history_list) >= 10: break

    valid_data_count = len(valid_dates_found)

    return location_name, target_date.strftime("%d%m%Y"), results, is_rookie_loc, last_visit_str, sales_history_list, advice_messages, valid_data_count, full_scan_list, efficiency_score

Overwriting core_logic.py


In [ ]:
%%writefile main.py
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
import glob
import os
import sys
import importlib
import config
importlib.reload(config)

from data_utils import parse_strict_date, load_json, load_sku_map
from core_logic import process_single_location_block
from storage_manager import update_archives_and_save

def generate_weekly_plan():
    print(f"📅 === Smart Weekly Scheduler ({config.VERSION}) ===\n")

    # ... (File loading logic remains same)
    today = datetime.datetime.now()
    today = today.replace(hour=0, minute=0, second=0, microsecond=0)
    start_of_week = today - timedelta(days=today.weekday())
    end_of_week = start_of_week + timedelta(days=6)

    xlsx_files = [f for f in os.listdir('.') if f.endswith('.xlsx') and "IMS" in f and not f.startswith('~$')]
    if not xlsx_files: return
    xlsx_files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
    target_file = xlsx_files[0]
    abs_file_path = os.path.abspath(target_file)
    sku_map = load_sku_map(abs_file_path)

    intervals = {}
    distributions = {}
    all_locations = set()

    xls = None
    try: xls = pd.ExcelFile(abs_file_path)
    except: return

    try:
        config_sheet = None
        for s in xls.sheet_names:
            if 'location' in s.lower(): config_sheet = s; break
        if config_sheet:
            ldf = pd.read_excel(xls, sheet_name=config_sheet, header=None)
            n_idx = -1; i_idx = -1; d_idx = -1
            for i, r in ldf.iterrows():
                row_str = str(r.values).lower()
                if 'location_id' in row_str: n_idx = i
                if 'interval' in row_str: i_idx = i
                if 'distribution' in row_str: d_idx = i
            if n_idx != -1 and i_idx != -1:
                names = ldf.iloc[n_idx]; vals = ldf.iloc[i_idx]
                dists = ldf.iloc[d_idx] if d_idx != -1 else None
                for c in range(len(ldf.columns)):
                    l_raw = names[c]
                    if pd.notna(l_raw) and str(l_raw).strip() != '' and str(l_raw).lower() != 'location_id':
                        l = str(l_raw).strip(); v = str(vals[c]).strip().lower()
                        all_locations.add(l)
                        d = 7
                        if v == '2': d = 14
                        elif v == '4': d = 28
                        elif '5x5' in v: d = 7
                        intervals[l] = d
                        dist_val = "Other"
                        if dists is not None and c < len(dists):
                            raw_dist = dists[c]
                            if pd.notna(raw_dist): dist_val = str(raw_dist).strip()
                        distributions[l] = dist_val
    except: pass

    print("🔍 Scanning Excel sheets and processing...")
    shadow = load_json(config.SHADOW_LEDGER_FILE)
    advisor_data = {}
    calculated_results = []
    found_locations = set()

    for sheet_name in xls.sheet_names:
        if "location" in sheet_name.lower() or "sku" in sheet_name.lower() or "sheet" in sheet_name.lower(): continue
        df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
        loc_indices = []
        for i, row in df.iterrows():
            if 'Location_Id' in " ".join([str(x) for x in row.values]): loc_indices.append(i)

        for i in range(len(loc_indices)):
            start_row = loc_indices[i]
            end_row = loc_indices[i+1] if i < len(loc_indices)-1 else len(df)
            blk = df.iloc[start_row:end_row].copy()
            loc_row = df.iloc[start_row]
            loc_name = None
            for col_idx, cell_val in enumerate(loc_row):
                if str(cell_val).strip() == 'Location_Id':
                    if col_idx + 1 < len(loc_row): loc_name = str(loc_row[col_idx+1]).strip()
                    break
            if not loc_name: continue
            found_locations.add(loc_name)

            date_row = df.iloc[start_row - 1] if start_row > 0 else loc_row
            last_d = None; valid_ds = []
            for c in range(len(df.columns)):
                d = parse_strict_date(date_row.iloc[c])
                if d: valid_ds.append(d)
            if valid_ds: last_d = max(valid_ds)
            if not last_d: continue

            inter = intervals.get(loc_name, config.DEFAULT_INTERVAL)
            due = last_d + timedelta(days=inter)
            status = ""; is_active = False

            # [v29.1] Overdue Logic Update:
            # If overdue, set repl date to "today", and pass as Override
            target_date_override = None

            if due < start_of_week:
                status = "🔴 Overdue"
                is_active = True
                due = datetime.datetime.now() # Correct to today
                target_date_override = due
            elif start_of_week <= due <= end_of_week:
                status = "🟢 Planned"
                is_active = True
                # Pass calculated date for consistency
                target_date_override = due
            else:
                status = "⚪️ Skipped"
                is_active = False

            ln, ld, res, is_new, last_visit, sales_hist, advice, data_count, full_list, eff_score = process_single_location_block(
                blk, loc_name, date_row, sku_map, {}, {}, shadow, advisor_data, current_interval=inter, dry_run=(not is_active),
                target_date_override=target_date_override # Pass param
            )

            calculated_results.append({
                'loc': ln, 'dist': distributions.get(ln, "Other"), 'status': status, 'date': due,
                'res': res if is_active else [], 'is_new': is_new, 'last_visit': last_visit,
                'sales_hist': sales_hist, 'advice': advice,
                'data_count': data_count, 'full_list': full_list,
                'eff_score': eff_score
            })

    dead_locations = all_locations - found_locations
    for dead_loc in dead_locations:
        calculated_results.append({'loc': dead_loc, 'dist': distributions.get(dead_loc, "Other"), 'status': "⚫️ Dead", 'date': datetime.datetime.now(), 'res': [], 'is_new': False, 'last_visit': "N/A", 'sales_hist': [], 'advice': [], 'data_count': "N/A", 'full_list': [], 'eff_score': None})

    if not calculated_results: return
    fname = f"Weekly_Plan_{start_of_week.strftime('%Y%m%d')}.xlsx"
    print(f"💾 Generating Excel Report: {fname}")

    try:
        with pd.ExcelWriter(fname, engine='xlsxwriter') as writer:
            summary_rows = []
            status_order = {"🔴": 0, "🟢": 1, "⚪️": 2, "⚫️": 3}
            calculated_results.sort(key=lambda x: status_order.get(x['status'][0], 99))

            for item in calculated_results:
                due_str = item['date'].strftime('%Y-%m-%d')
                if "Dead" in item['status']: due_str = "N/A"

                eff = item.get('eff_score')
                eff_str = "N/A (Rookie)"
                waste_str = "-"
                missed_str = "-"
                rating_str = "-"

                if eff:
                    eff_str = f"{eff['score_pct']:.0%}"
                    waste_str = f"{eff['waste']:.0f}"
                    missed_str = f"{eff['missed']:.1f}"
                    rating_str = eff['rating']

                row_data = {
                    'Distribution': item['dist'], 'Status': item['status'], 'Location': item['loc'],
                    'Last Visit': item['last_visit'], 'Due Date': due_str,
                    'Rookie Mode': 'Yes' if item['is_new'] else 'No',
                    'Efficiency': eff_str,
                    'Over-Carry': waste_str,
                    'Under-Carry': missed_str,
                    'Rating': rating_str
                }
                for i in range(10):
                    val = item['sales_hist'][i] if i < len(item['sales_hist']) else ""
                    row_data[f'Daily Sales (T-{i+1})'] = val
                summary_rows.append(row_data)
            pd.DataFrame(summary_rows).to_excel(writer, sheet_name="Weekly Overview", index=False)

            # Format Overview Page
            ws_sum = writer.sheets["Weekly Overview"]
            ws_sum.set_column('A:A', 15); ws_sum.set_column('B:B', 20); ws_sum.set_column('C:C', 25)
            ws_sum.set_column('G:J', 12)

            dist_groups = {}
            for item in calculated_results:
                if not item['res']: continue
                raw_dist = str(item['dist']).strip(); d_name = raw_dist.title() if raw_dist else "Other"
                if d_name not in dist_groups: dist_groups[d_name] = []
                dist_groups[d_name].append(item)

            count_created = 0
            fmt_header = writer.book.add_format({'bold': True, 'font_color': 'blue', 'font_size': 14})
            fmt_col_header = writer.book.add_format({'bold': True, 'bottom': 1})
            fmt_warning = writer.book.add_format({'font_color': 'red', 'bold': True})
            fmt_full_title = writer.book.add_format({'bold': True, 'bg_color': '#E0E0E0'})

            for d_name, items in dist_groups.items():
                sheet_title = str(d_name)[:30].replace('/','').replace('[','').replace(']','')
                if sheet_title == "": sheet_title = "Other"
                worksheet = writer.book.add_worksheet(sheet_title)
                writer.sheets[sheet_title] = worksheet
                current_col = 0

                for item in items:
                    header_text = f"{item['loc']}\nDue: {item['date'].strftime('%Y-%m-%d')}"
                    worksheet.merge_range(0, current_col, 0, current_col+2, header_text, fmt_header)

                    df_res = pd.DataFrame(item['res'], columns=['Item', 'Qty', 'Cap'])
                    worksheet.write(1, current_col, "Item", fmt_col_header)
                    worksheet.write(1, current_col+1, "Qty", fmt_col_header)
                    worksheet.write(1, current_col+2, "Cap", fmt_col_header)

                    last_data_row = 2
                    for row_num, row_data in enumerate(df_res.values):
                        worksheet.write(2 + row_num, current_col, row_data[0])
                        worksheet.write(2 + row_num, current_col+1, row_data[1])
                        worksheet.write(2 + row_num, current_col+2, row_data[2])
                        last_data_row = 2 + row_num

                    if item['advice']:
                        worksheet.write(last_data_row + 2, current_col, "💡 Advice:", fmt_warning)
                        for adv_idx, advice_text in enumerate(item['advice']):
                            worksheet.write(last_data_row + 3 + adv_idx, current_col, advice_text)
                        last_data_row += len(item['advice']) + 3
                    else: last_data_row += 3

                    full_start_row = last_data_row + 2
                    worksheet.write(full_start_row, current_col, "📋 Full Check", fmt_full_title)
                    worksheet.write(full_start_row + 1, current_col, "Item", fmt_col_header)
                    worksheet.write(full_start_row + 1, current_col+1, "Qty", fmt_col_header)
                    worksheet.write(full_start_row + 1, current_col+2, "Cap", fmt_col_header)

                    df_full = pd.DataFrame(item['full_list'], columns=['Item', 'Qty', 'Cap'])
                    for row_num, row_data in enumerate(df_full.values):
                        worksheet.write(full_start_row + 2 + row_num, current_col, row_data[0])
                        worksheet.write(full_start_row + 2 + row_num, current_col+1, row_data[1])
                        worksheet.write(full_start_row + 2 + row_num, current_col+2, row_data[2])

                    worksheet.set_column(current_col, current_col, 30)
                    worksheet.set_column(current_col+1, current_col+1, 6)
                    worksheet.set_column(current_col+2, current_col+2, 6)
                    worksheet.set_column(current_col+3, current_col+3, 2)
                    current_col += 4
                    count_created += 1

        update_archives_and_save(shadow, advisor_data)
        print(f"✅ Success! Report generated: {fname}")

    except Exception as e:
        print(f"❌ Error generating Excel: {e}")

if __name__ == '__main__':
    generate_weekly_plan()

Overwriting main.py


In [ ]:
!python main.py

📅 === Smart Weekly Scheduler (v29.1 (Feat: Overdue Auto-Correction)) ===

✅ SKU Map loaded successfully: Contains 120 products
🔍 Scanning Excel sheets and processing...
💾 Generating Excel Report: Weekly_Plan_20260105.xlsx
✅ All background archives safely updated (Shadow, History, Graveyard, Visit).
✅ Success! Report generated: Weekly_Plan_20260105.xlsx
